<a href="https://colab.research.google.com/github/SeHongPark-96/naver_news_crawling/blob/main/naver_global_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
import requests

import warnings
warnings.filterwarnings('ignore')

import os
import selenium
from selenium import webdriver
import re
from datetime import datetime as dt, timedelta
import datetime
from tqdm import tqdm

import pandas as pd
import time
import math
import numpy as np

from dateutil.parser import parse
import pymssql
from selenium.webdriver import Chrome, ChromeOptions
import lxml
import lxml.html
import schedule
import time

In [ ]:
yesterday = (datetime.date.today()-timedelta(1)).strftime('%Y%m%d')
yesterday

'20210825'

### 텍스트 전처리 함수

In [ ]:
def cleansing(text):
            text = text.replace('// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}',"")
        
            text = re.sub('([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})', '', text)
            text = re.sub("[\n\t]", '', text)
            text = re.sub(r"\[.*\]|\s-\s.*", "", text) # [ ] 와 사이의 문자 제거



            text = re.sub("(=*([가-힣]{2,4}|[가-힣]{6}) 기자(... [가-힣]{2,3}기자|[^가-힣]|$)=*|로봇 기자)",r' 기자\3 ',text) # 000기자 -> 기자로 변환
            text = re.sub('(‘|\")(.+)(’|\")',r" '\2' ", text)                     # ' " ’ -> 로  변환
            text = re.sub('【(.+)】',r' [\1] ', text)                                #【( 등과 같은 괄호 '[ ]' 로 변환
            text = re.sub('(http|ftp|https)://((\w+\.|\w+/|\w+|(\?|=|\&)\w+)+)', ' 주소 ', text)  #url -> '주소' 로 변환
            text = re.sub('[0-9]+-[0-9]+|02-[0-9]+-[0-9]+',' 번호 ', text)        #000-0000-0000 -> '번호'로 변환
            text = re.sub('TOP [0-9]+|TOP[0-9]+',' 순위', text)                    # TOP 1, TOP 10 -> '순위'로 변환
            text = re.sub('[0-9]+:[0-9]+|[0-9]+시 [0-9]분',' 시간 ', text)         # 12:12 , 12시 12 분 -> '시간'로 변환
            text = re.sub('코로나19|코로나 19',' 코로나 ', text)                   # 코로나19, 코로나 19 -> '코로나' 로 변환
            text = re.sub('(\[\w+)(=)(\w+\])',r'\1 \2 \3',text )                # 해럴드경제(대전)=홍길동기자 ->해럴드경제 (대전) 홍길동기자        
            text = re.sub("(\w+)(\")(\w+)",r"\1 \2 \3",text)                    
            text = re.sub('(&nbsp;)|(&nbsp)|(&rsquo;)|(&rsquo)|(&lsquo;)|(&lsquo)|(&middot;)|(&middot)|(&#[0-9]+;)|(fffff;)|\x03|\n',"",text) # css 공통 문법 제거
            text = re.sub('(<([^>]+)>)', '', text)                              # html 문법 제거하기 태그포함
            text = re.sub("[']", '', text)
            text = re.sub('quot|rdquo|ldquo', '"', text)
            text = re.sub('[-=+,#/▶▲●◆■◀\?.:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》☞△&·]', ' ', text) # 특문제거
            re.findall('\d+', text)

            if re.search('[a-zA-Z]+-[a-zA-Z]+(:|;)',text)!=None:
                text = " ".join(re.findall('[가-힣]+|[0-9]+[가-힣]+',text)) #css 문법(font-family 등)이 있는 경우에는 한글/숫자+한글인 것만 추출
            
            text = re.sub('\;', "", text)
            text = re.sub('[-_+]|돋움|&[a-zA-Z]{2,100};'," ",text)          # 돋움이라는 font가 들어있는 경우는 css에서 제거하기 힘들어서 별도로 제거 + &quot같은 것들도 제거
            text = re.sub('(\w+)(\s\.\.+|\.\.+)',"",text)     # ... , ..  -> 공백으로 변환
            text = re.sub('\s\s+'," ",text)                                # 공백 2개이상 -> 한개로

            if re.search('\S',text)==None:                                          # 텍스트가 비어있으면 '빈칸'로 채움
                text = 'blank' 

            return text

In [2]:
# 어제자 뉴스 크롤링

def global_news_crawling(date=yesterday):

    # 크롬 옵션
    op = ChromeOptions()
    op.add_argument('--headless')
    op.add_argument('--no-sandbox')
    op.add_argument('--disable-dev-shm-usage')

    browser = Chrome('C:/Users/user/Desktop/News_Crawling/chromedriver_win32/chromedriver.exe', options=op)
    ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'
    
    url = "https://news.naver.com/main/list.naver"

    header ={

        # 브라우저에 맞게 수정

        # "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        # "accept-encoding": "gzip, deflate, br",
        # "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        # "cache-control": "max-age=0",
        # "sec-ch-ua-mobile": "?0",
        # "sec-fetch-dest": "document",
        # "sec-fetch-mode": "navigate",
        # "sec-fetch-site": "same-origin",
        # "sec-fetch-user": "?1",
        # "upgrade-insecure-requests": "1",
        # "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

    }
    
    global_info = pd.DataFrame()
    
    # 경제 부문만
    category='글로벌 경제'


    # 마지막 페이지 번호 구하기
    for first_page in range(1,1000, 10):

        # category 마다 다르게 지정해줘야 함
        param = {
            "mode" : "LS2D",
            "mid" : "shm",
            "sid2" : "262",
            "sid1" : "101",
            "date" : f"{date}",
            "page" : f"{first_page}"
        }


        # 들어갈 링크 가져오기
        response = requests.get(url,params=param, headers=header).text
        soup = BeautifulSoup(response, 'html.parser')

        # "다음" 버튼이 있는지 없는지로 판단
        pre_btn = soup.find('a', class_='pre nclicks(fls.page)')
        next_btn = soup.find('a', class_="next nclicks(fls.page)")

        if next_btn != None:
            continue

        else:
            try:
                last_page = int(soup.find_all("a", class_="nclicks(fls.page)")[-1].text)
                print(f"{category} {date} 일자 최종 페이지 번호는 {last_page}입니다")                
                break

            except:
                paging_div = soup.find('div', class_='paging')
                last_page = int(paging_div.find_all('strong')[-1].text)
                print(f"{category} {date} 일자 최종 페이지 번호는 {last_page}입니다")                
                break

        time.sleep(0.5)

    # 마지막 페이지까지 기사별 링크 가져오기
    all_news_links = []

    for page in range(1, last_page+1): #last_page+1

        param = {
            "mode" : "LS2D",
            "mid" : "shm",
            "sid1" : "101",
            "sid2" : "262",
            "date" : f"{date}",
            "page" : f"{page}"
        }

        # 아무런 기사가 없는 날에도 page는 1로 존재하는 경우에 대한 처리

        try:
            # 들어갈 링크 가져오기
            response = requests.get(url,params=param, headers=header).text
            soup = BeautifulSoup(response, 'html.parser')


            # 이미지가 있는 경우 하나의 기사당 모두 3개의 동일한 href를 가짐 - class명 등 구분 방법이 따로 없어 모두 가져온 후 set() 처리
            article_table = soup.find("div", class_="list_body newsflash_body") # link명이 기사용과 다름
            page_news_links = article_table.find_all("a")


            for page_news_link in page_news_links:
                all_news_links.append(page_news_link.attrs['href'])

            time.sleep(0.5)

        except:
            break

    all_news_links = set(all_news_links)

    print("기사별 링크 크롤링 완료")

    time.sleep(1)


    # 실제 기사 속 내용 모두 가져오기


    for i, news_link in enumerate(tqdm(all_news_links)) :

        # 기사 속 내용 가져오기
        article_response = requests.get(news_link, headers=header).text
        article_soup = BeautifulSoup(article_response, 'html.parser')
        
        browser.get(news_link)
        time.sleep(0.5)
        
        html = browser.page_source
        root = lxml.html.fromstring(html)
        

        # 잘못된 url이 있는 경우가 존재했음 - pass로 제외하고 진행
        try:               
            news_title = article_soup.find('h3', id='articleTitle').text
            news_title = re.sub(r"\[.*\]|\s-\s.*", "", news_title)

            news_date = article_soup.find('span', class_='t11').text

            url_detail = article_soup.find('a', class_='btn_artialoriginal').attrs['href']
            contents = article_soup.find('div', class_="_article_body_contents").text
    #                     article = re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\n\t]", '', article_soup.find('div', class_="_article_body_contents").text)
    #                     article = re.sub(' +', ' ', article).lstrip()
            contents = contents.replace('// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}',"")

        # 좋아요 / 화나요 크롤링 코드 추가
            good_body = root.cssselect('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.good > a > span.u_likeit_list_count._count')
            good = good_body[0].text_content()
            bad_body = root.cssselect('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.angry > a > span.u_likeit_list_count._count')
            bad = bad_body[0].text_content()
        

            global_info = global_info.append(pd.DataFrame([[category, news_title, news_date, url_detail, contents, good, bad]], columns=['CATEGORY', 'TITLE', 'NEWS_DATE', 'URL_DETAIL', 'clean_content', 'good', 'bad'])).reset_index(drop=True)
            time.sleep(0.5)
            # article_info = article_info.append(pd.DataFrame([[good, bad]], columns=['good', 'bad'])).reset_index(drop=True)

        except:
            pass

    # 형식 추가 (날짜 형식 수정 + 날짜별 내림차순 + 날짜와 순번을 결합한 IDX 추가)
    global_info['NEWS_DATE'] = global_info['NEWS_DATE'].apply(lambda x: x.replace('오후','PM').replace('오전','AM'))
    global_info['NEWS_DATE'] = global_info['NEWS_DATE'].apply(lambda x: datetime.datetime.strptime(x,"%Y.%m.%d. %p %I:%M"))



    global_info = global_info.sort_values('NEWS_DATE',ascending=False).reset_index(drop=True)
    global_info['IDX'] = [date.strftime('%y%m%d') + '{0:04d}'.format(i+1) for i,date in enumerate(global_info['NEWS_DATE'])]
    global_info['clean_content'] = global_info['clean_content'].apply(lambda x: cleansing(x))

    global_info = global_info[['IDX','CATEGORY','NEWS_DATE','TITLE','clean_content','URL_DETAIL','good', 'bad']]





    global_info.to_csv(f'C:/Users/user/Desktop/News_Crawling/global_article/global_info_{date}.csv', encoding='utf-8-sig', index=False)    

In [ ]:
global_news_crawling()

글로벌 경제 20210825 일자 최종 페이지 번호는 6입니다
기사별 링크 크롤링 완료


100%|████████████████████████████████████████████████████████████████████████████████| 118/118 [03:15<00:00,  1.66s/it]


In [ ]:
articles = pd.read_csv(f"C:/Users/user/Desktop/News_Crawling/global_article/global_info_{yesterday}.csv")
articles['clean_content'] = articles['clean_content'].apply(lambda x: cleansing(x))
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   IDX            118 non-null    int64 
 1   CATEGORY       118 non-null    object
 2   NEWS_DATE      118 non-null    object
 3   TITLE          118 non-null    object
 4   clean_content  118 non-null    object
 5   URL_DETAIL     118 non-null    object
 6   good           118 non-null    int64 
 7   bad            118 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 7.5+ KB


In [ ]:
if articles['good'].dtype=='O' and articles['bad'].dtype=='O':
    articles['good'] = articles['good'].apply(lambda x: int(x.replace(',', '')))
    articles['bad'] = articles['bad'].apply(lambda x: int(x.replace(',', '')))
    
elif articles['good'].dtype=='O' and articles['bad'].dtype!='O':
    articles['good'] = articles['good'].apply(lambda x: int(x.replace(',', '')))
    
elif articles['good'].dtype!='O' and articles['bad'].dtype=='O':
    articles['bad'] = articles['bad'].apply(lambda x: int(x.replace(',', '')))

articles.fillna('', inplace=True)

articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   IDX            118 non-null    int64 
 1   CATEGORY       118 non-null    object
 2   NEWS_DATE      118 non-null    object
 3   TITLE          118 non-null    object
 4   clean_content  118 non-null    object
 5   URL_DETAIL     118 non-null    object
 6   good           118 non-null    int64 
 7   bad            118 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 7.5+ KB
